# Calcular la Agregación de Opporunity

In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import os
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
# plt.style.use('ggplot')
%matplotlib inline
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [63]:
from src.data.location_notebook import get_location
import src.features.build_features as bf

In [64]:
(DATA_DIR, DATA_DIR_PROCESSED, 
INTERMEDIATE_RESULTS, GROUP_INTERMEDIATE_RESULTS, 
CLEANED_DS , TRAIN_RAW_FILE, 
VALIDATION_RAW_FILE, TEMPLATE_SUBMIT) = get_location()

In [65]:
df_train = pd.read_parquet(INTERMEDIATE_RESULTS)

## Agregación
- El dataset tiene varios registros para cada oportunidad. 
- se reagrupará la oportunidad con las variables que se modifican entre los grupos
    - para ello, vamos a buscar las columnas que cambian al reagrupar
- Luego de realizar la agregación, tendremos una sola fila con:
    - todos los productos que compró
    - todos las familias de productos que compró
    - Planned_Range mean y std
    - Total_Amount acumulado

In [66]:
# buscamos las columnas que cambian
oc = df_train.columns[~df_train.columns.isin(np.append(prod_cols, [ "Opportunity_ID" ]))].to_numpy() 
for c in oc:
    if (df_train[['Opportunity_ID', c]].drop_duplicates().shape[0] != df_train.Opportunity_ID.nunique()): print(f"Agrupar por: {c}")

Agrupar por: TRF
Agrupar por: Planned_Range_days
Agrupar por: Planned_Delivery_Start_Date_Period
Agrupar por: Planned_Delivery_Start_Date_Year
Agrupar por: Planned_Delivery_Start_Date_Month
Agrupar por: Planned_Delivery_Start_Date_Q1_Quarter
Agrupar por: Planned_Delivery_Start_Date_Q2_Quarter
Agrupar por: Planned_Delivery_Start_Date_Q3_Quarter
Agrupar por: Planned_Delivery_Start_Date_Q4_Quarter
Agrupar por: Planned_Delivery_End_Date_Period
Agrupar por: Planned_Delivery_End_Date_Year
Agrupar por: Planned_Delivery_End_Date_Month
Agrupar por: Planned_Delivery_End_Date_Q1_Quarter
Agrupar por: Planned_Delivery_End_Date_Q2_Quarter
Agrupar por: Planned_Delivery_End_Date_Q3_Quarter
Agrupar por: Planned_Delivery_End_Date_Q4_Quarter
Agrupar por: Planned_Delivery_End_Date_Imputed


In [67]:
# (df_train.groupby('Opportunity_ID').ASP_was_converted.nunique() > 1).reset_index().groupby('ASP_was_converted').Opportunity_ID.unique().iloc[1]
# df_train.loc[df_train.Opportunity_ID == 1694,["Total_Amount_Currency", "ASP_was_converted", "rate"]]

In [68]:
# (df_train.groupby('Opportunity_ID').Total_Amount_Imputed.nunique() > 1).reset_index().groupby('Total_Amount_Imputed').Opportunity_ID.unique().iloc[1]
# df_train.loc[df_train.Opportunity_ID == 1694,["Total_Amount_Currency", "ASP_was_converted", "rate"]]

In [69]:
df_train = bf.product_agg(df_train)

In [70]:
df_train.to_parquet(GROUP_INTERMEDIATE_RESULTS, index=False)

_________________
### Legacy Code 

In [113]:
# from tqdm import tqdm

In [114]:
# agg_cols = [ 
#     "Total_Amount", 
#     "TRF",
#     "Planned_Range_days_mean",
#     "Planned_Range_days_std",

#     "Planned_Delivery_Start_Date_Period_mean",
#     "Planned_Delivery_Start_Date_Period_std",
#     "Planned_Delivery_Start_Date_Year_mean",
#     "Planned_Delivery_Start_Date_Year_std",
#     "Planned_Delivery_Start_Date_Month_mean",
#     "Planned_Delivery_Start_Date_Month_std",
#     "Planned_Delivery_Start_Date_Quarter_mean", # ?
#     "Planned_Delivery_Start_Date_Quarter_std", # ?
    
#     "Planned_Delivery_End_Date_Period_mean",
#     "Planned_Delivery_End_Date_Period_std",
#     "Planned_Delivery_End_Date_Year_mean",
#     "Planned_Delivery_End_Date_Year_std",
#     "Planned_Delivery_End_Date_Month_mean",
#     "Planned_Delivery_End_Date_Month_std",
#     "Planned_Delivery_End_Date_Quarter_mean", # ?
#     "Planned_Delivery_End_Date_Quarter_std", # ?
# ]

# agg_planned_cols = [
#     "Planned_Range_days",
#     "Planned_Delivery_Start_Date_Period",
#     "Planned_Delivery_Start_Date_Year",
#     "Planned_Delivery_Start_Date_Month",
#     "Planned_Delivery_Start_Date_Quarter",
#     "Planned_Delivery_End_Date_Period",
#     "Planned_Delivery_End_Date_Year",
#     "Planned_Delivery_End_Date_Month",
#     "Planned_Delivery_End_Date_Quarter",
# ]
    
    

In [115]:
# def aggregate_data(df, prod_cols):
#     res = pd.DataFrame([], columns=np.append(prod_cols, agg_cols + ["Opportunity_ID"]))
#     accum = []
#     for key, group in tqdm(df.groupby('Opportunity_ID')):
#         s = group[np.append(prod_cols, [ "Total_Amount", "TRF" ])].sum()
        
#         planned_range = group.agg({c:["std", "mean"] for c in agg_planned_cols})
#         for c in agg_planned_cols:
#             s = s.append(pd.Series(planned_range[[c]].loc["mean"].fillna(0).iloc[0], index=[c+"_mean"]))
#             s = s.append(pd.Series(planned_range[[c]].loc["std"].fillna(0).iloc[0], index=[c+"_std"]))

#         s = s.append(pd.Series([key], index=["Opportunity_ID"]))
#         accum.append(s)
#     res = res.append(accum, ignore_index=True)
#     return res



si es la primera vez que se ejecutan, quitar el script false, ya que es una operación que demanda mucho tiempo

In [116]:
# %%script false --no-raise-error
# agg_train = aggregate_data(df_train, prod_cols)
# agg_val = aggregate_data(df_val, prod_cols)

In [117]:
# %%script false --no-raise-error
# este en el caso que ya se haya ejecutado, leer el resultado
# agg_train = pd.read_parquet(os.path.join(DATA_DIR,"processed/intermediate_group_result_train.parquet"))
# agg_val = pd.read_parquet(os.path.join(DATA_DIR,"processed/intermediate_group_result_val.parquet"))

In [118]:
# agg_train.shape, agg_val.shape